https://www.youtube.com/watch?v=aRaGHNZeVkI&list=PLjxUCNDRYw8k1_HTzXDohUHKhYKYFQrbn&index=2

Standard Imports

In [1]:
# LabOne Q:
from laboneq.simple import *

# Helpers:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_results
from laboneq.contrib.example_helpers.generate_example_datastore import (
    generate_example_datastore,
    get_first_named_entry,
)

from pathlib import Path
import time

Load device setup for MQV Fraunhofer

In [3]:

#DESCRITPTOR 
#Define initial PHYSICAL device setup, Logical Signal Lines of the Instrument are defined here and later in a Signal Map mapped to the Experimental Signal lines 

#Fill in missing address

descriptor_setup_small= """\
instruments:
  SHFSG:
  - address: DEV12265
    uid: device_shfsg_1
  SHFQA:
  - address: DEV12249
    uid: device_shfqa_1
  PQSC:
  - address: DEV10----
    uid: device_pqsc
connections:
  device_shfsg:
    - iq_signal: q0/drive_line
      ports: SGCHANNELS/0/OUTPUT
  device_shfqa:
    - iq_signal: q0/measure_line
      ports: QACHANNELS/0/OUTPUT
    - acquire_signal: q0/acquire_line
      ports: QACHANNELS/0/INPUT
  device_pqsc:
    - to: device_shfsg_1
      port: ZSYNCS/0
    - to: device_shfqa_1
      port: ZSYNCS/1
"""


# define the DeviceSetup from descriptor - additionally include information on the dataserver used to connect to the instruments 
#with the device set
device_setup = DeviceSetup.from_descriptor(
    #or with example import and yaml also possible: yaml_text=descriptor_shfsg_shfqa_pqsc
    descriptor_setup_small,
    server_host="127.0.0.1", #IP-Address of the LabOne (NOT Q) dataserver used to communicate with the instrument
    server_port="8004",
    setup_name="psi",
) 
emulate = False


In [4]:
# create and connect to a session
session = Session(device_setup=device_setup)
session.connect(do_emulation=emulate)

[2023.10.09 10:34:57.674] INFO    Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is u:\LabOneQ Software Local Samuel\laboneq\LabPC_Software\Qubit_Characterization\laboneq_output\log


AttributeError: 'NoneType' object has no attribute 'uid'

Experiment Parameters

In [ ]:
# frequency range of spectroscopy scan - around expected centre frequency as defined in qubit parameters
start_freq = -200.0e6
stop_freq = 200.0e6
num_points = 101
integration_time = 1e-3

# define number of averages
# used for 2^num_averages, maximum: num_averages = 17
num_averages = 4


# define sweep parameter
def create_readout_freq_sweep(qubit, start_freq, stop_freq, num_points):
    return LinearSweepParameter(
        uid=f"{qubit}_res_freq",
        start=start_freq,
        stop=stop_freq,
        count=num_points,
        axis_name="Frequency [Hz]",
    )

create a function which generates you CW spectroscopy experiment. In this experiment, you'll pass the LinearSweepParameter defined previously as an argument to the sweep section. Within the section, you'll create a section containing an acquire command.

In [ ]:
# function that defines a resonator spectroscopy experiment, and takes the frequency sweep as a parameter
def res_spectroscopy_CW(freq_sweep, exp_settings):
    # Create resonator spectroscopy experiment - uses only readout drive and signal acquisition
    exp_spec = Experiment(
        uid="Resonator Spectroscopy",
        signals=[
            ExperimentSignal("measure"),
            ExperimentSignal("acquire"),
        ],
    )

    ## define experimental sequence
    # loop - average multiple measurements for each frequency - measurement in spectroscopy mode
    with exp_spec.acquire_loop_rt(
        uid="shots",
        count=pow(2, exp_settings["num_averages"]),
        acquisition_type=AcquisitionType.SPECTROSCOPY,
    ):
        with exp_spec.sweep(uid="res_freq", parameter=freq_sweep):
            # readout pulse and data acquisition
            with exp_spec.section(uid="spectroscopy"):
                # resonator signal readout
                exp_spec.acquire(
                    signal="acquire",
                    handle="res_spec",
                    length=exp_settings["integration_time"],
                )
            with exp_spec.section(uid="delay", length=1e-6):
                # holdoff time after signal acquisition
                exp_spec.reserve(signal="measure")
                exp_spec.reserve(signal="acquire")

    return exp_spec

Experiment Calibration and Signal Map

In [ ]:
# function that returns the calibration of the readout line oscillator for the experimental signals
def res_spec_calib(freq_sweep):
    exp_calibration = Calibration()
    # sets the oscillator of the experimental measure signal
    exp_calibration["measure"] = SignalCalibration(
        # for spectroscopy, use the hardware oscillator of the QA, and set the sweep parameter as frequency
        oscillator=Oscillator(
            "readout_osc",
            frequency=freq_sweep,
            modulation_type=ModulationType.HARDWARE,
        )
    )
    return exp_calibration


# signal maps for the two different qubits - maps the logical signal of the device setup to the experimental signals of the experiment


def res_spec_map(qubit):
    signal_map = {
        "measure": device_setup.logical_signal_groups[f"{qubit}"].logical_signals[
            "measure_line"
        ],
        "acquire": device_setup.logical_signal_groups[f"{qubit}"].logical_signals[
            "acquire_line"
        ],
    }
    return signal_map


# define the experiment with the frequency sweep relevant for qubit 0
freq_sweep = create_readout_freq_sweep("q0", start_freq, stop_freq, num_points)
exp_settings = {"integration_time": integration_time, "num_averages": num_averages}
exp_spec = res_spectroscopy_CW(freq_sweep, exp_settings)

# set signal calibration and signal map for experiment to qubit 0
exp_spec.set_calibration(res_spec_calib(freq_sweep))
exp_spec.set_signal_map(res_spec_map("q0"))

Compile and Generate Pulse Sheet

In [ ]:
# compile the experiment on the open instrument session
compiled_res_spec = session.compile(exp_spec)

Path("Pulse_Sheets").mkdir(parents=True, exist_ok=True)
# generate a pulse sheet to inspect experiment before runtime
show_pulse_sheet("Pulse_Sheets/Resonator_Spectroscopy_Pulse_Sheet", compiled_res_spec)

Run, Save and Plot Results

In [ ]:
# run the compiled experiemnt
res_spec_results = session.run(compiled_res_spec)
timestamp = time.strftime("%Y%m%dT%H%M%S")
Path("Results").mkdir(parents=True, exist_ok=True)
session.save_results(f"Results/{timestamp}_results.json")
print(f"File saved as Results/{timestamp}_results.json")

In [ ]:
# plot the results
plot_results(res_spec_results, phase=True)